In [80]:
import torchvision
from torchvision import transforms
import torch
import torch.utils.data as data
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import joblib
import mlflow

In [81]:
mlflow.set_tracking_uri('http://127.0.0.1:8080')
mlflow.set_experiment('pytorch')

<Experiment: artifact_location='mlflow-artifacts:/976142275983112277', creation_time=1741888125188, experiment_id='976142275983112277', last_update_time=1741888125188, lifecycle_stage='active', name='pytorch', tags={}>

In [82]:
# Parameters
resize_fact = 128
batch_size = 20
num_epochs = 50
modelname = 'model4'
learning_rate = 0.001

In [83]:

transforms_list = transforms.Compose([
    transforms.Resize((resize_fact,resize_fact)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

train_data = torchvision.datasets.ImageFolder(root='../data/train/',transform=transforms_list)
val_data = torchvision.datasets.ImageFolder(root='../data/val/',transform=transforms_list)
test_data = torchvision.datasets.ImageFolder(root='../data/test/',transform=transforms_list)

train_data_loader = data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_data_loader = data.DataLoader(val_data,batch_size=batch_size,shuffle=True)
test_data_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

In [84]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(resize_fact*resize_fact*3,84)
        self.fc2 = nn.Linear(84,50)
        self.fc3 = nn.Linear(50,2)
    def forward(self,x):
        x = x.view(-1,resize_fact*resize_fact*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # Since we are going to use CrossEntropyLoss, we don't need to apply softmax here, bcoz it is already applied in the loss function
        return x

In [85]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(resize_fact*resize_fact*3,200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,2)
    def forward(self,x):
        x = x.view(-1,resize_fact*resize_fact*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x) # Since we are going to use CrossEntropyLoss, we don't need to apply softmax here, bcoz it is already applied in the loss function
        return x

In [86]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(resize_fact*resize_fact*3,200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,2)
    def forward(self,x):
        x = x.view(-1,resize_fact*resize_fact*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x,0.5)
        x = F.relu(self.fc3(x))
        x = self.fc4(x) # Since we are going to use CrossEntropyLoss, we don't need to apply softmax here, bcoz it is already applied in the loss function
        return x

In [87]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(resize_fact*resize_fact*3,300)
        self.fc2 = nn.Linear(300,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,2)
    def forward(self,x):
        x = x.view(-1,resize_fact*resize_fact*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))
        x = self.fc4(x) # Since we are going to use CrossEntropyLoss, we don't need to apply softmax here, bcoz it is already applied in the loss function
        return x

In [88]:
if modelname=='model1':
    model=Model1()
elif modelname=='model2':
    model=Model2()
elif modelname=='model3':
    model=Model3()
elif modelname=='model4':
    model=Model4()

In [89]:
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
model.to(device)

Model4(
  (fc1): Linear(in_features=49152, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=2, bias=True)
)

In [90]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
def train_model(model, optimizer,loss_fun,train_loader,val_loader,epochs=20,device='gpu'):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs,targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fun(outputs,targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        
        training_loss /= len(train_loader.dataset)
        writer.add_scalar('Loss/train',training_loss,epoch)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs,targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fun(outputs,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(outputs,dim=1),dim=1)[1],targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        val_accuracy= num_correct/num_examples
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch,training_loss,valid_loss,num_correct/num_examples))
    return model,val_accuracy


In [91]:
trained_model,val_accuracy = train_model(model, optimizer,nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=num_epochs, device=device)

Epoch: 0, Training Loss: 0.84, Validation Loss: 0.70, accuracy = 0.75
Epoch: 1, Training Loss: 0.42, Validation Loss: 0.90, accuracy = 0.61
Epoch: 2, Training Loss: 0.34, Validation Loss: 0.56, accuracy = 0.71
Epoch: 3, Training Loss: 0.23, Validation Loss: 0.47, accuracy = 0.79
Epoch: 4, Training Loss: 0.17, Validation Loss: 0.89, accuracy = 0.75
Epoch: 5, Training Loss: 0.12, Validation Loss: 0.74, accuracy = 0.79
Epoch: 6, Training Loss: 0.17, Validation Loss: 0.97, accuracy = 0.64
Epoch: 7, Training Loss: 0.16, Validation Loss: 0.89, accuracy = 0.75
Epoch: 8, Training Loss: 0.27, Validation Loss: 1.38, accuracy = 0.75
Epoch: 9, Training Loss: 0.22, Validation Loss: 0.66, accuracy = 0.68
Epoch: 10, Training Loss: 0.21, Validation Loss: 1.02, accuracy = 0.71
Epoch: 11, Training Loss: 0.06, Validation Loss: 1.42, accuracy = 0.71
Epoch: 12, Training Loss: 0.12, Validation Loss: 1.22, accuracy = 0.71
Epoch: 13, Training Loss: 0.09, Validation Loss: 1.13, accuracy = 0.79
Epoch: 14, Train

In [92]:
loss_fun = nn.CrossEntropyLoss()
valid_loss=0.0
num_correct = 0
num_examples = 0
with torch.no_grad():
    for batch in test_data_loader:
        inputs,targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = trained_model(inputs)
        loss = loss_fun(outputs,targets)
        valid_loss += loss.data.item() * inputs.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs,dim=1),dim=1)[1],targets).view(-1)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    valid_loss /= len(test_data_loader.dataset)

In [93]:
print('Val Accuracy: {:.2f}'.format(val_accuracy))
test_accuracy = num_correct/num_examples
print('Test Accuracy: {:.2f}'.format(test_accuracy))

Val Accuracy: 0.79
Test Accuracy: 0.75


In [94]:
with mlflow.start_run():
    mlflow.log_param('resize_fact',resize_fact)
    mlflow.log_param('batch_size',batch_size)
    mlflow.log_param('num_epochs',num_epochs)
    mlflow.log_param('modelname',modelname)
    mlflow.log_param('learning_rate',learning_rate)
    mlflow.log_metric('val_accuracy',val_accuracy)
    mlflow.log_metric('test_accuracy',test_accuracy)
    mlflow.pytorch.log_model(trained_model,'model')
    joblib.dump(trained_model,'../models/model.pkl')

2025/03/13 19:14:31 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.2.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


2025/03/13 19:14:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run trusting-whale-381 at: http://127.0.0.1:8080/#/experiments/976142275983112277/runs/361ad0a67d504c40b70e79854337c722
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/976142275983112277
